#### This notebook fits an Recurring Neural Network over Headlines dataset for sarcasm detection.

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import hstack
from normalizer import Normalizer
import tensorflow as tf
from tensorflow import keras

tf.device = tf.device("gpu")

First let's set our envoringment to GPU!

Load the dataset:

In [36]:
data = pd.read_json("../dataset/Sarcasm_Headlines_Dataset.json", lines = True)

Explore the data:

In [37]:
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_link  26709 non-null  object
 1   headline      26709 non-null  object
 2   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 626.1+ KB


Check if the datatset is balanced?

In [39]:
len(data.loc[data["is_sarcastic"]==0])

14985

In [40]:
len(data.loc[data["is_sarcastic"]==1])

11724

The dataset is unbalanced let's balance it!

In [41]:
sarcastic_data = data.loc[data["is_sarcastic"]==1][:500]
nonsarcastic_data = data.loc[data["is_sarcastic"]==0][:500]

data = pd.concat([sarcastic_data, nonsarcastic_data])
data.head(5)

,article_link,headline,is_sarcastic
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
8,https://politics.theonion.com/top-snake-handle...,top snake handler leaves sinking huckabee camp...,1
15,https://entertainment.theonion.com/nuclear-bom...,nuclear bomb detonates during rehearsal for 's...,1
16,https://www.theonion.com/cosby-lawyer-asks-why...,cosby lawyer asks why accusers didn't come for...,1


Now the dataset is balanced.

In [42]:
data = data.drop(columns=["article_link"])

Seperate the labels from the text.

In [43]:
X = data["headline"]
y = data["is_sarcastic"]

Now using the Normalization package that we created before, let's vectorize the text:

In [44]:
X_matrix = Normalizer().vectorize(pd.DataFrame({"headline": X}))
X_matrix

[INFO] Trying to create a sparse matrix for text, using an instance of TfIdf_vectorizer
[INFO] Extracting columns containing text from dataframe.
[INFO] Successfully extracted text columns from the dataset.
headline
[INFO] Applying Normalization over text:
[INFO]       - Converting Text into lower case for caseconsistency.
[INFO]       - Extracting only words containing alphabets.
[INFO] Text Normalization is now complete.
[INFO] Fitting the vecotirzer to given text.
[INFO] Transforming the text into a sparse matrix.
[INFO] Sparse Matrix has been successfully created over the text given as input.


<1000x4182 sparse matrix of type '<class 'numpy.float64'>'
	with 9478 stored elements in Compressed Sparse Row format>

Let's split the set into validation and train set:

In [45]:
X_train, X_Test, y_train, y_test = train_test_split(X_matrix, y, test_size=0.33, random_state=42)

Now that we have the necessary datasets ready let's fit an RNN to it.

In [46]:

rnn = keras.Sequential([
    keras.layers.LSTM(128, activation='relu', input_shape=(X_train.shape[1], 1), return_sequences=True),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])


# Compile the RNN model
rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [47]:
history = rnn.fit(X_train.toarray(), y_train, epochs=10, batch_size=512)


Epoch 1/10


2/2 [==============================] - 128s 95s/step - loss: 0.6932 - accuracy: 0.4880
Epoch 2/10
2/2 [==============================] - 198s 132s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 232s 137s/step - loss: 0.6931 - accuracy: 0.5059
Epoch 4/10
2/2 [==============================] - 256s 147s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 269s 152s/step - loss: 0.6932 - accuracy: 0.5000
Epoch 6/10
2/2 [==============================] - 282s 159s/step - loss: 0.6932 - accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 281s 155s/step - loss: 0.6932 - accuracy: 0.5000
Epoch 8/10
2/2 [==============================] - 280s 154s/step - loss: 0.6932 - accuracy: 0.5000
Epoch 9/10
2/2 [==============================] - 284s 156s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 10/10
2/2 [==============================] - 288s 159s/step - loss: 0.6931 - accuracy: 0.5000
